<a href="https://colab.research.google.com/github/shouvikcirca/PCB_Defect_Detection/blob/master/VGG19_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import keras 
from keras.applications import VGG19
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [ ]:
#######################

In [3]:
conv_base = VGG19(weights='imagenet', include_top = False, input_shape=(300,300,3))

80142336/80134624 [==============================] - 1s 0us/step


In [2]:
from google.colab import drive
drive.mount("/content/gdrive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [4]:
import pickle
X = np.load("gdrive/My Drive/pcb/xtrain.npy")
y = np.load("gdrive/My Drive/pcb/ytrain.npy")


dataset_size = X.shape[0]

In [ ]:
# Training set has 472 samples

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [8]:
nos = X_train.shape[0]
nos

377

In [9]:
((y==1).sum())/((y==0).sum()),((y_train==1).sum())/((y_train==0).sum()),((y_test==1).sum())/((y_test==0).sum())

(1.0, 0.9635416666666666, 1.1590909090909092)

In [10]:
y_test.shape, (y_test==1).sum(), (y_test==0).sum()

((95,), 51, 44)

In [11]:
# datagen = ImageDataGenerator(rescale=1./255, )
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

In [12]:
datagen.fit(X_train)

In [13]:
bs = 34
numit = NumpyArrayIterator(
    X_train, y_train, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

In [14]:
conv_base.predict(X_train[0].reshape(1,300,300,3)).shape


(1, 9, 9, 512)

In [15]:
train_samples = np.zeros(shape=(nos,9,9,512))
train_labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    train_samples[i*bs:(i+1)*bs] = f
    train_labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(34, 9, 9, 512)
(34, 9, 9, 512)
(34, 9, 9, 512)
(34, 9, 9, 512)
(34, 9, 9, 512)
(34, 9, 9, 512)
(34, 9, 9, 512)
(34, 9, 9, 512)
(34, 9, 9, 512)
(34, 9, 9, 512)
(34, 9, 9, 512)
(3, 9, 9, 512)


In [16]:
del X_train    # deleting X_train to make space

In [18]:
train_samples = train_samples.reshape(nos,9*9*512)
train_samples.shape, train_labels.shape

((377, 41472), (377,))

In [19]:
nostest = X_test.shape[0]
nostest

95

In [20]:
datagentest = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

In [21]:
datagentest.fit(X_test)

In [22]:
numitest = NumpyArrayIterator(
    X_test, y_test, datagentest, batch_size=nostest, shuffle=True, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


test_samples = np.zeros(shape=(nostest,9,9,512))
test_labels = np.zeros(shape=(nostest))
for samples_batch,labels_batch in numitest:
    g = conv_base.predict(samples_batch)
    print(g.shape)
    test_samples[::] = g
    test_labels[:] = labels_batch
    break

(95, 9, 9, 512)


In [23]:
del X_test

In [25]:
test_samples = test_samples.reshape(nostest,9*9*512)
test_samples.shape

(95, 41472)

In [26]:
from keras import models, layers, optimizers
model = models.Sequential()
model.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation = 'sigmoid'))


model.compile(optimizer = optimizers.Adam(lr = 1e-3),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])
        


In [27]:
my_callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),

]

In [57]:
history = model.fit(train_samples, train_labels, epochs = 100, batch_size = 20,
                    validation_data = (test_samples, test_labels), callbacks = my_callbacks
)


Train on 377 samples, validate on 95 samples
Epoch 1/100
377/377 [==============================] - 0s 1ms/step - loss: 0.1581 - accuracy: 0.9363 - val_loss: 0.6015 - val_accuracy: 0.7263
Epoch 2/100
377/377 [==============================] - 0s 1ms/step - loss: 0.1369 - accuracy: 0.9443 - val_loss: 0.6062 - val_accuracy: 0.7684
Epoch 3/100
377/377 [==============================] - 0s 1ms/step - loss: 0.1115 - accuracy: 0.9602 - val_loss: 0.5274 - val_accuracy: 0.7579
Epoch 4/100
377/377 [==============================] - 0s 977us/step - loss: 0.0878 - accuracy: 0.9629 - val_loss: 0.6326 - val_accuracy: 0.7684
Epoch 5/100
377/377 [==============================] - 0s 990us/step - loss: 0.1037 - accuracy: 0.9629 - val_loss: 0.5816 - val_accuracy: 0.7579
Epoch 6/100
377/377 [==============================] - 0s 1ms/step - loss: 0.0980 - accuracy: 0.9576 - val_loss: 0.7000 - val_accuracy: 0.7474
Epoch 7/100
377/377 [==============================] - 0s 974us/step - loss: 0.0684 - accurac

In [ ]:
# run = 1      22:100

In [58]:
preds = model.predict(test_samples, batch_size = 20)
predclass = np.array([1.0 if i>0.5 else 0.0 for i in preds])
(predclass == test_labels).sum()/95

0.7578947368421053

In [59]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_labels,predclass)

array([[33, 11],
       [12, 39]])

In [60]:
confusion_matrix(test_labels,predclass)[0][0]/44, confusion_matrix(test_labels,predclass)[1][1]/51

(0.75, 0.7647058823529411)

In [ ]:
test_labels.shape

(534,)

Validation Data

In [37]:
xval = np.load('gdrive/My Drive/pcb/xtest.npy')
yval = np.load('gdrive/My Drive/pcb/ytest.npy')


In [38]:
datagenval = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

In [39]:
datagenval.fit(xval)

In [40]:
numitval = NumpyArrayIterator(
    xval, yval, datagenval, batch_size=yval.shape[0], shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


val_samples = np.zeros(shape=(yval.shape[0],9,9,512))
val_labels = np.zeros(shape=(yval.shape[0]))
for samples_batch,labels_batch in numitval:
    g = conv_base.predict(samples_batch)
    print(g.shape)
    val_samples[::] = g
    val_labels[:] = labels_batch
    break

(552, 9, 9, 512)


In [41]:
val_samples = val_samples.reshape(yval.shape[0],9*9*512)

In [61]:
val_preds = model.predict(val_samples, batch_size = 20)

In [62]:
val_predclass = np.array([1.0 if i>0.5 else 0.0 for i in val_preds])

In [63]:
(val_predclass == val_labels).sum()/552

0.7644927536231884

In [64]:
confusion_matrix(val_labels,val_predclass)

array([[ 46,  13],
       [117, 376]])

In [65]:
confusion_matrix(val_labels,val_predclass)[0][0]/59, confusion_matrix(val_labels,val_predclass)[1][1]/493

(0.7796610169491526, 0.7626774847870182)

In [ ]:
# model_json = model.to_json()
# with open("gdrive/My Drive/models/VGG19_12.json", "w") as json_file:
#     json_file.write(model_json)

In [ ]:
# model.save_weights("gdrive/My Drive/models/VGG19_12.h5")
# print("Saved model to disk")

Saved model to disk


In [ ]:
# Saving and loading models
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/